In [62]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error,mean_squared_log_error,r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, weight_boosting, BaggingRegressor
from sklearn.neighbors import  KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn import tree
import statsmodels.api as sm
import sklearn.preprocessing as sk
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [63]:
sns.set()
warnings.filterwarnings('ignore')

In [64]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
dfs = [train, test]
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [65]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [66]:
train.columns = [x.lower() for x in train.columns]
test.columns = [x.lower() for x in test.columns]

In [67]:
train.saleprice.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: saleprice, dtype: float64

In [68]:
def get_nulls(df):
    nulls = pd.DataFrame(df.isnull().sum().sort_values(ascending=False))[df.isnull().sum().sort_values() > 0]
    nulls.columns = ['Null Count']
    nulls['Null Pct'] = round(nulls['Null Count']/df.shape[0],4)
    nulls.index.name = 'Feature'
    return(nulls)

def get_null_colct(df):
    nulls = pd.DataFrame(df.isnull().sum().sort_values(ascending=False))[df.isnull().sum().sort_values() > 0]
    nulls.columns = ['Null Count']
    nulls['Null Pct'] = round(nulls['Null Count']/df.shape[0],4)
    nulls.index.name = 'Feature'
    return(len(nulls))

def get_null_colnames(df):
    nulls = pd.DataFrame(df.isnull().sum().sort_values(ascending=False))[df.isnull().sum().sort_values() > 0]
    nulls.columns = ['Null Count']
    nulls['Null Pct'] = round(nulls['Null Count']/df.shape[0],4)
    nulls.index.name = 'Feature'
    return(list(nulls.index))

get_nulls(train)

,Null Count,Null Pct
Feature,,
poolqc,1453,0.9952
miscfeature,1406,0.9630
alley,1369,0.9377
fence,1179,0.8075
fireplacequ,690,0.4726
lotfrontage,259,0.1774
garagecond,81,0.0555
garagetype,81,0.0555
garageyrblt,81,0.0555


In [69]:
get_null_colnames(train)

['poolqc',
 'miscfeature',
 'alley',
 'fence',
 'fireplacequ',
 'lotfrontage',
 'garagecond',
 'garagetype',
 'garageyrblt',
 'garagefinish',
 'garagequal',
 'bsmtexposure',
 'bsmtfintype2',
 'bsmtfintype1',
 'bsmtcond',
 'bsmtqual',
 'masvnrarea',
 'masvnrtype',
 'electrical']

In [70]:
def encode(x): return 0 if pd.isna(x) else 1
train['enc_alley'] = train.alley.apply(encode)
test['enc_alley'] = train.alley.apply(encode)

In [71]:
def encode(x): return 0 if pd.isna(x) else 1
train['enc_fence'] = train.fence.apply(encode)
test['enc_fence'] = train.fence.apply(encode)

In [72]:
train = train.drop(columns = ['poolqc', 'miscfeature', 'alley', 'fence', 'garageyrblt'], axis = 0)
test = test.drop(columns = ['poolqc', 'miscfeature', 'alley', 'fence', 'garageyrblt'], axis = 0)

In [73]:
get_nulls(train)

,Null Count,Null Pct
Feature,,
fireplacequ,690,0.4726
lotfrontage,259,0.1774
garagecond,81,0.0555
garagetype,81,0.0555
garagefinish,81,0.0555
garagequal,81,0.0555
bsmtexposure,38,0.0260
bsmtfintype2,38,0.0260
bsmtfintype1,37,0.0253


In [74]:
def encode(x): return 0 if pd.isna(x) else x
train['lotfrontage'] = train.lotfrontage.apply(encode)
test['lotfrontage'] = train.lotfrontage.apply(encode)

In [75]:
def encode(x): return 'None' if pd.isna(x) else x
train.fireplacequ = test.fireplacequ.apply(encode)
test.fireplacequ = test.fireplacequ.apply(encode)

In [76]:
get_nulls(train)

,Null Count,Null Pct
Feature,,
garagecond,81,0.0555
garagetype,81,0.0555
garagefinish,81,0.0555
garagequal,81,0.0555
bsmtfintype2,38,0.0260
bsmtexposure,38,0.0260
bsmtfintype1,37,0.0253
bsmtcond,37,0.0253
bsmtqual,37,0.0253


In [77]:
sts = train.describe().transpose()

In [78]:
train.fillna(sts['mean'],inplace=True)
test.fillna(sts['mean'],inplace=True)

In [79]:
get_nulls(train)

,Null Count,Null Pct
Feature,,
garagecond,81,0.0555
garagetype,81,0.0555
garagefinish,81,0.0555
garagequal,81,0.0555
bsmtfintype2,38,0.0260
bsmtexposure,38,0.0260
bsmtfintype1,37,0.0253
bsmtcond,37,0.0253
bsmtqual,37,0.0253


In [80]:
cat_list = list(train.dtypes[train.dtypes == 'object'].index)
print('# of categoricals: {}'.format(len(cat_list)))

# of categoricals: 39


In [81]:
data = pd.concat([train,test],axis=0)

In [82]:
for x in cat_list:
     if (data[x].isna().sum() > 0) :
        data[x].fillna(max(data[x].mode()), inplace=True)

In [83]:
get_nulls(data)

,Null Count,Null Pct
Feature,,
saleprice,1459,0.4998


In [84]:
data = pd.get_dummies(data = data, drop_first=True)
train = data.iloc[:1460,:]
test = data.iloc[1460:,:]

In [85]:
target = train.pop('saleprice')
target = np.log(target)

In [86]:
get_nulls(train)

,Null Count,Null Pct
Feature,,


In [87]:
XX = train.copy()
y = target.copy()
history= pd.DataFrame(columns=['itreation','feature_cat','pvalue','R_adjusted','rsquared'])

for x in range(len(XX.columns)):
        regressor_osl=sm.OLS(endog=y,exog=XX).fit()
        pvalues=regressor_osl.pvalues
        pvalues=pd.DataFrame(pvalues)
        p=pd.DataFrame(pvalues.reset_index())
        out=pd.DataFrame()
        out=p.loc[p[0]== max(p[0])]
        out=pd.Series.tolist(out)
        w1=out[0][0]
        R=regressor_osl.rsquared_adj
        r=regressor_osl.rsquared
        XX.pop(w1)
        history=history.append([{'itreation':x+1,'feature_cat':out[0][0],'pvalue':out[0][1],'R_adjusted':R,'rsquared':r}])

In [88]:
drop = history[history['pvalue']>.05]['feature_cat']
drop = list(drop)
drop

['neighborhood_BrkSide',
 'electrical_FuseF',
 'bldgtype_Duplex',
 'garagefinish_RFn',
 'miscval',
 'exterqual_Gd',
 'exterior2nd_BrkFace',
 'paveddrive_P',
 'neighborhood_Timber',
 'roofstyle_Hip',
 'neighborhood_SWISU',
 'exterior1st_Stucco',
 'exterior1st_ImStucc',
 'lotfrontage',
 'housestyle_1.5Unf',
 'exterior1st_Stone',
 'neighborhood_NPkVill',
 'exterior1st_AsphShn',
 'housestyle_SLvl',
 'bsmtfintype2_GLQ',
 'bsmtfintype1_BLQ',
 'lowqualfinsf',
 'condition1_RRNe',
 'exterior2nd_Stone',
 'condition2_RRNn',
 'roofstyle_Gable',
 'saletype_ConLw',
 'condition2_Norm',
 'bldgtype_TwnhsE',
 'functional_Min2',
 'exterior2nd_Other',
 'mosold',
 'bsmtexposure_Mn',
 'salecondition_Partial',
 'bsmthalfbath',
 'exterior2nd_CBlock',
 'exterior1st_CBlock',
 'exterqual_Fa',
 'neighborhood_Gilbert',
 'neighborhood_SawyerW',
 'exterior1st_MetalSd',
 'masvnrarea',
 'bsmtunfsf',
 'neighborhood_Blueste',
 'exterqual_TA',
 'housestyle_SFoyer',
 'bsmtfintype1_Rec',
 'salecondition_Family',
 'extercon

In [89]:
train.drop(columns = drop, inplace = True, axis = 0)
test.drop(columns = drop, inplace = True, axis = 0)

In [91]:
train.head()

,bsmtfinsf1,bsmtfullbath,enc_fence,enclosedporch,fireplaces,garagearea,garagecars,grlivarea,kitchenabvgr,lotarea,...,roofmatl_CompShg,roofmatl_Membran,roofmatl_Metal,roofmatl_Roll,roofmatl_Tar&Grv,roofmatl_WdShake,roofmatl_WdShngl,salecondition_Normal,saletype_ConLD,saletype_New
0,706.0,1.0,0,0,0,548.0,2.0,1710,1,8450,...,1,0,0,0,0,0,0,1,0,0
1,978.0,0.0,0,0,1,460.0,2.0,1262,1,9600,...,1,0,0,0,0,0,0,1,0,0
2,486.0,1.0,0,0,1,608.0,2.0,1786,1,11250,...,1,0,0,0,0,0,0,1,0,0
3,216.0,1.0,0,272,1,642.0,3.0,1717,1,9550,...,1,0,0,0,0,0,0,0,0,0
4,655.0,1.0,0,0,1,836.0,3.0,2198,1,14260,...,1,0,0,0,0,0,0,1,0,0


In [32]:
history[history['pvalue']<.05].sort_values('rsquared', ascending = False)[0:29]

,itreation,feature_cat,pvalue,R_adjusted,rsquared
0,159,housestyle_2Story,0.048084,0.999924,0.999929
0,165,condition1_RRNn,0.042664,0.999923,0.999927
0,166,bsmtfintype2_BLQ,0.040660,0.999923,0.999927
0,167,enc_fence,0.029859,0.999923,0.999927
0,168,garagecond_Po,0.033174,0.999923,0.999927
0,169,condition1_Feedr,0.045124,0.999923,0.999926
0,170,condition1_RRAn,0.032922,0.999923,0.999926
0,171,condition1_PosN,0.037494,0.999922,0.999926
0,172,exterior2nd_Wd Sdng,0.036024,0.999922,0.999926
0,174,lotconfig_Inside,0.032573,0.999922,0.999925


In [232]:
pre_proc = [preprocessing.Normalizer, preprocessing.MaxAbsScalar, preprocessing.MinMaxScalar, preprocessing.KernelCenterer, preprocessing.StandardScaler]

def pre_proc_loop(X_train, X_test):
    for proc in pre_proc:
     X_train = proc.fit_transform(X_train)
     X_test = prc.transform(X_test)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
        ..
1429   NaN
1430   NaN
1431   NaN
1432   NaN
1433   NaN
1434   NaN
1435   NaN
1436   NaN
1437   NaN
1438   NaN
1439   NaN
1440   NaN
1441   NaN
1442   NaN
1443   NaN
1444   NaN
1445   NaN
1446   NaN
1447   NaN
1448   NaN
1449   NaN
1450   NaN
1451   NaN
1452   NaN
1453   NaN
1454   NaN
1455   NaN
1456   NaN
1457   NaN
1458   NaN
Name: saleprice, Length: 1459, dtype: float64

In [227]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100, test_size = .2)

Index(['bsmtfinsf1', 'bsmtfullbath', 'enclosedporch', 'fireplaces',
       'garagearea', 'garagecars', 'grlivarea', 'lotarea', 'overallcond',
       'overallqual', 'screenporch', 'totalbsmtsf', 'wooddecksf', 'yearbuilt',
       'yearremodadd', 'yrsold', 'bsmtexposure_Gd', 'bsmtexposure_No',
       'centralair_Y', 'condition1_Feedr', 'exterior1st_BrkComm',
       'exterior1st_BrkFace', 'functional_Typ', 'kitchenqual_TA',
       'mszoning_RL', 'mszoning_RM', 'neighborhood_Crawfor',
       'neighborhood_Edwards', 'salecondition_Normal', 'saletype_New'],
      dtype='object')

In [221]:
sel = SelectFromModel(GradientBoostingRegressor(n_estimators = 100), threshold = -np.inf, max_features = 30)
sel.fit(train, target)

SelectFromModel(estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
        max_features=30, norm_order=1, prefit=False, threshold=-inf)

In [222]:
selected_features = train.columns[(sel.get_support())]
X = train[selected_features]
print(selected_features)
print('\n Number of Features: {}'.format(len(selected_features)))

Index(['bsmtfinsf1', 'bsmtfullbath', 'enclosedporch', 'fireplaces',
       'garagearea', 'garagecars', 'grlivarea', 'lotarea', 'overallcond',
       'overallqual', 'screenporch', 'totalbsmtsf', 'wooddecksf', 'yearbuilt',
       'yearremodadd', 'yrsold', 'bsmtexposure_Gd', 'bsmtexposure_No',
       'centralair_Y', 'condition1_Feedr', 'exterior1st_BrkComm',
       'exterior1st_BrkFace', 'functional_Typ', 'kitchenqual_TA',
       'mszoning_RL', 'mszoning_RM', 'neighborhood_Crawfor',
       'neighborhood_Edwards', 'salecondition_Normal', 'saletype_New'],
      dtype='object')

 Number of Features: 30


In [223]:
gbm_model = GradientBoostingRegressor(n_estimators = 100, random_state = 1)
gbm_model.fit(X, target)
gbm_model.score(X, target)

0.9513852763612697

In [141]:
# Number of trees in random forest.
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]

# Number of features to consider at every split.
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree.
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node.
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node.
min_samples_leaf = [1, 2, 4]


# Create the random grid.
parm_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(parm_grid)

{'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [142]:
#gbm_random = RandomizedSearchCV(estimator = gbm_model, 
#                             param_distributions = parm_grid,
#                               n_iter = 100,
#                               cv = 3,
#                               verbose=0,
#                               random_state= 1,
#                               n_jobs = -1)
#gbm_random.fit(X, target)
#gbm_random.best_params_

In [615]:
lm = linear_model.LinearRegression()
lm.fit(X_train,  y_train)

y_pre=lm.predict(X_train)
r2_score(y_train,y_pre)

0.9404059888698137

In [449]:
lm.score(X_test, y_test)

0.8419408921271484

In [443]:
predictions = lm.predict(X_test)
mean_squared_error(y_test, predictions)

0.02721168991441709